In [1]:
import numpy as np
import sacc
import matplotlib.pyplot as plt
import pyccl as ccl
import scipy
import importlib as imp

import CM_code.spurious_george as sp
imp.reload(sp)
import CM_code.halo_model as halo
imp.reload(halo)
import CM_code.lsst_coZmology as zed
imp.reload(zed)

# import main.py from TJPCov folder
import TJPCov.tjpcov as tjp

In [2]:
# import LSST SRD cosmology from CM modules
cosmo = zed.cosmo_SRD

# import frequently used values from spurious george
th_min = sp.theta_min
th_max = sp.theta_max
th_bins = sp.theta_cents
N_bins = sp.N_bins

d2r = np.pi / 180.

### Let's start by creating $C_{gg}^{l}$

In [3]:
ell = np.unique(np.geomspace(180/(sp.theta_max/60.) - 10, 
                             180/(sp.theta_min/60.) + 10, 
                             1024).astype(int))

# compute angular power spectrum for galaxy-galaxy lensing
Cl_gg, lensTracer, sourceTracer = sp.get_LSST_Cl(ell, 'gg')

### Next we set all the parameter data that TJP will need

In [5]:
Cgg_data = sacc.Sacc()
Cgg_data.metadata['info'] = 'Cl_gg for LSST-like lenses and sources'

Cgg_data.metadata['fsky'] = sp.pa.fsky # fsky from Danielle's LSST params

# TJP wants theta in DEGREES
Cgg_data.metadata['th_bins'] = th_bins/60.

# WHAT IS GOING ON HERE??? - Q. still stands, didn't have time to talk about it
th=np.logspace(np.log10(th_min*0.98),np.log10(1.),N_bins*30)
th2=np.linspace(1.,th_max*1.02,N_bins*30) 
Cgg_data.metadata['th'] = np.unique(np.sort(np.append(th,th2)))

# Add ell values to meta data
Cgg_data.metadata['ell'] = ell
Cgg_data.metadata['ell_bins'] = np.linspace(ell[0],ell[-1],60)

thb=0.5*(Cgg_data.metadata['th_bins'][1:]+Cgg_data.metadata['th_bins'][:-1])

### Now carry out the first part of the calculation, Wigner Transform

In [6]:
# Need to define spin of tracers for Wigner Transform
# CHECK WITH DANIELLE: Both tracers are position tracers so spin 0?
WT_factors={}
WT_factors['lens','source']=(0,0)
WT_factors['source','lens']=(0,0)
WT_factors['lens','lens']=(0,0)
WT_factors['source','source']=(0,0)

In [13]:
# Set kwargs for Wigner Transform (fourier space transform so need theta and ell)
WT_kwargs={'l': Cgg_data.metadata['ell'],
           'theta': Cgg_data.metadata['th']*d2r, 's1_s2':[(0,0),(0,0),(0,0),(0,0)]}
%time WT = tjp.wigner_transform(**WT_kwargs)

CPU times: user 728 ms, sys: 1.94 s, total: 2.67 s
Wall time: 3.52 s


In [16]:
# Define tracer noise (should be same for both as they are position tracers)
Ngal = 3600. * n_s # Check this definition for Ngal
tracer_noise = 1./ Ngal

### Finally, try compute the covariance matrix for Cl_gg
- Need to figure out what the correct tracer combinations are

    Ok, so we have: *lens-lens, source-lens, and source-source*

In [48]:
# import redshift data and set K values of Pk calculations
zl, dndz_l, zleff = zed.get_dndz_spec('lens')
zs, dndz_s, zseff = zed.get_dndz_spec('source')

a_l = 1. / (1. + zl)
a_s = 1. / (1. + zs)

k = np.geomspace(1E-4, 5E2, 3*sp.interp_len)

In [49]:
pk_ll, pk_ls, pk_ss = halo.get_Pgg_2D(k, np.flip(a_l))

pk_ll done...
pk_ls done...
pk_ss done...
returning: Pk_ll, Pk_ls, Pk_ss


In [77]:
#tracer_combs = [[lensTracer,lensTracer],[lensTracer,sourceTracer],[sourceTracer,sourceTracer]]
tracer_combs = [['lensTracer','lensTracer'],['lensTracer','sourceTracer'],['sourceTracer','sourceTracer']]
tracer_comb1 = tracer_combs[0]
tracer_comb2 = tracer_combs[2]

pk_arr = np.full((2, 2), fill_value=[[pk_ll, pk_ll], [pk_ll, pk_ll]])

for i in range(2):
    for j in range(2):
        print(tracer_comb1[i],tracer_comb2[j])
        if tracer_comb1[i] == lensTracer and tracer_comb2[j] == lensTracer:
            pk_arr[i,j] = pk_ll
            print('position %d%d = pk_ll'%(i,j))
        elif tracer_comb1[i] == sourceTracer and tracer_comb2[j] == sourceTracer:
            pk_arr[i,j] = pk_ss
            print('position %d%d = pk_ss'%(i,j))
        else:
            pk_arr[i,j] = pk_ls
            print('position %d%d = pk_ls'%(i,j))

lensTracer sourceTracer
position 00 = pk_ls
lensTracer sourceTracer
position 01 = pk_ls
lensTracer sourceTracer
position 10 = pk_ls
lensTracer sourceTracer
position 11 = pk_ls


In [57]:
# Next we can compute Clgg for each tracer combination
ell = Cgg_data.metadata['ell']
Cgg = {} 
    
Cgg[13] = ccl.angular_cl(cosmo, tracer_comb1[0], tracer_comb2[0], ell)
Cgg[24] = ccl.angular_cl(cosmo, tracer_comb1[1], tracer_comb2[1], ell)
Cgg[14] = ccl.angular_cl(cosmo, tracer_comb1[0], tracer_comb2[1], ell)
Cgg[23] = ccl.angular_cl(cosmo, tracer_comb1[1], tracer_comb2[0], ell)

norm = (2*ell+1)*np.gradient(ell)*Cgg_data.metadata['fsky']



NameError: name 'tracer_comb1' is not defined